# Explorar acesso ao kafka

In [1]:
!pip install --quiet azure-storage-blob delta

In [2]:
import sys
import requests
import time
import os
import pyspark
# from distutils.log import ERROR
from azure.storage.blob import BlobClient
from delta import *
from os import path
# from matplotlib import container
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StringType, DateType, StructType, StructField
from pyspark.sql.avro.functions import *
import socket

In [3]:
spark = (SparkSession.builder
    .appName("ExploreKafka")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")         
    .getOrCreate()
)
spark.sparkContext.setLogLevel("DEBUG")

In [4]:
socket.gethostbyname("nemesys-sr-schema-registry-headless.nemesys-stream-101.svc")

'10.2.0.11'

In [5]:
bootstrap = "nemesys-stream-101-kafka-bootstrap.nemesys-stream-101.svc:9093"
offset = "earliest"
topico = "stocks.StockData.stocks"
SCHEMA_REGISTRY = "http://nemesys-sr-schema-registry-headless.nemesys-stream-101.svc:8081"

In [6]:
# retrieve the latest schema
url = '{}/subjects/{}-value/versions/latest/schema'.format(SCHEMA_REGISTRY, topico)
# print(url)
response = requests.get(url)
# error check
response.raise_for_status()
# extract the schema from the response
schema = response.text

In [7]:
(spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", bootstrap)
    .option("subscribe", topico)
    .option("startingOffsets", offset)
    .load()
    .selectExpr("substring(value, 6) as avro_value")
    .select(from_avro(col("avro_value"), schema).alias("value"))
    .select("value.*")
    .writeStream
    .format('delta')
    .outputMode('append')
    .option('mergeSchema', 'true')
    .option('checkpointLocation', "lakehouse/checkpoint/stocks")
    .trigger(once=True)
    .start("lakehouse/stocks")
    .awaitTermination()
)
    

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df = (
    spark
    .read
    .format("delta")
    .load("lakehouse/stocks")
)

In [ ]:
df.toPandas()